In [17]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Percorsi dei file Excel
f1 = r"C:\Users\milan\OneDrive\Documenti\difensori_2019_reordered.xlsx"
f2 = r"C:\Users\milan\OneDrive\Documenti\difensori_2020_reordered.xlsx"
f3 = r"C:\Users\milan\OneDrive\Documenti\difensori_2021_reordered.xlsx"
f4 = r"C:\Users\milan\OneDrive\Documenti\difensori_2022_reordered.xlsx"
f5 = r"C:\Users\milan\OneDrive\Documenti\difensori_2023_reordered.xlsx"

# Colonne da eliminare
colonne_da_eliminare = ['Player', 'Squad', 'Nation', 'Pos', 'short Cmp%', 'medium Cmp%', 'long Cmp%', 'AerTot']

# Leggi tutti i file Excel
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2).iloc[1:].reset_index(drop=True)
df3 = pd.read_excel(f3).iloc[1:].reset_index(drop=True)
df4 = pd.read_excel(f4).iloc[1:].reset_index(drop=True)
df5 = pd.read_excel(f5).iloc[1:].reset_index(drop=True)

# Uniforma tutte le colonne basandoti su df1
colonne_comuni = df1.columns.drop(colonne_da_eliminare)  # Prendi tutte le colonne di df1, tranne quelle da eliminare

# Funzione per uniformare le colonne di ogni DataFrame
def uniforma_colonne(df, colonne_riferimento):
    # Mantieni solo le colonne presenti nel riferimento
    df = df[[col for col in colonne_riferimento if col in df.columns]]
    # Aggiungi eventuali colonne mancanti con valori NaN
    for col in colonne_riferimento:
        if col not in df.columns:
            df[col] = np.nan
    # Ordina le colonne nello stesso ordine del riferimento
    df = df[colonne_riferimento]
    return df

# Applica la funzione a tutti i DataFrame
df1 = uniforma_colonne(df1, colonne_comuni)
df2 = uniforma_colonne(df2, colonne_comuni)
df3 = uniforma_colonne(df3, colonne_comuni)
df4 = uniforma_colonne(df4, colonne_comuni)
df5 = uniforma_colonne(df5, colonne_comuni)

# Unisci i DataFrame degli anni precedenti
df_unito = pd.concat([df1, df2, df3, df4], ignore_index=True)

# X prende tutte le colonne tranne 'Tkl' e 'Blocks'
X = df_unito.drop(columns=['Tkl', 'Blocks'])

# y prende solo le colonne 'Tkl' e 'Blocks'
y = df_unito[['Tkl', 'Blocks']]

# Pre-elaborazione
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.select_dtypes(include=[np.number]))  # Solo colonne numeriche

# Divisione in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Costruzione della rete neurale
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1])
])

# Compilazione del modello
model.compile(optimizer='adam', loss='mean_squared_error')

# Addestramento del modello
model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.2)

# Pre-elabora i dati nuovi (df5)
X_new = df5.drop(columns=['Tkl', 'Blocks'])

# Assicurati che le colonne siano numeriche e nel giusto ordine
X_new = X_new[colonne_comuni.drop(['Tkl', 'Blocks'])].select_dtypes(include=[np.number])

# Normalizza i dati nuovi
X_new_scaled = scaler.transform(X_new)

# Usa il modello per fare previsioni
y_pred = model.predict(X_new_scaled)

# Imposta a 0 tutti i valori predetti che sono minori di 0
y_pred = np.clip(y_pred, 0, None)  # Sostituisci i valori negativi con 0

# Visualizza le previsioni
df5['Pred_Tkl'] = y_pred[:, 0]  # Previsioni per la prima colonna (Tkl)
df5['Pred_Blocks'] = y_pred[:, 1]  # Previsioni per la seconda colonna (Blocks)

# Stampa le colonne desiderate
print(df5[['Tkl', 'Blocks', 'Pred_Tkl', 'Pred_Blocks']])

# Calcola separatamente la somma delle previsioni per Gls e Ast
somma_tackle_attesi = df_new['Pred_Tkl'].sum()
somma_blocks_attesi = df_new['Pred_Blocks'].sum()

# Calcola separatamente la somma effettiva dei goal e assist nei dati originali
somma_tackle = df_new['Tkl'].sum()
somma_blocks = df_new['Blocks'].sum()

Epoch 1/500


c:\Users\milan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 682.3752 - val_loss: 730.5768
Epoch 2/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 599.8057 - val_loss: 638.2430
Epoch 3/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 542.0898 - val_loss: 471.1616
Epoch 4/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 377.8257 - val_loss: 247.9616
Epoch 5/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 196.2929 - val_loss: 134.3372
Epoch 6/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 126.5453 - val_loss: 115.3148
Epoch 7/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 110.5035 - val_loss: 100.1366
Epoch 8/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 92.0698 - val_loss: 87.2826
Epoch 9/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 83.1930 - val_loss: 76.8605
Epoch 10/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 63.6935 - val_loss: 68.0928
Epoch 11/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 51.6168 - val_loss: 61.2597
Epoch 12/500
18/18 ━━━━━━━━━━━━━━━━━━━

In [20]:
# Risultati dei tackle
print(f"La somma dei tackle effettivi è {somma_tackle}")
print(f"La somma dei tackle ottenuti dalla rete neurale è {somma_tackle_attesi}")

# Risultati delle intercettazioni

print(f"La somma delle intercettazioni effettivi è {somma_blocks}")
print(f"La somma degli intercettazioni ottenuti dalla rete neurale è {somma_blocks_attesi}")

# Errore

diff_tkl = abs(somma_tackle - somma_tackle_attesi)
diff_blk = abs(somma_blocks - somma_blocks_attesi)

# Trovo l'errore medio dividendo per il numero di giocatori 

avg_err_tkl = diff_tkl/194  
avg_err_blk = diff_blk/194

print(f"La differenza fra la somma effettiva e quella della rete neurale è di {diff_tkl} Tackles")
print(f"La differenza fra la somma effettiva e quella della rete neurale è di {diff_blk} Intercettazioni")
print(f"L'errore medio per i tackles è {avg_err_tkl} per giocatore")
print(f"L'errore medio per le intercettazioni è {avg_err_blk} per giocatore")

La somma dei tackle effettivi è 5220
La somma dei tackle ottenuti dalla rete neurale è 5139.31982421875
La somma delle intercettazioni effettivi è 3972
La somma degli intercettazioni ottenuti dalla rete neurale è 3749.24609375
La differenza fra la somma effettiva e quella della rete neurale è di 80.68017578125 Tackles
La differenza fra la somma effettiva e quella della rete neurale è di 222.75390625 Intercettazioni
L'errore medio per i tackles è 0.4158771947487113 per giocatore
L'errore medio per le intercettazioni è 1.148216011597938 per giocatore
